In [1]:
# Enforces PEP-8 style guideline
!pip install --upgrade pip
!pip install flake8 pycodestyle_magic

# Adds upper path to import common module
import sys
sys.path.append('../../')

/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Version 1: TIMEOUT
- used methods: DP

"""
from collections import namedtuple
from functools import lru_cache

INF = 1234567890

Item = namedtuple('Item', ['name', 'volume', 'urgency'])


# Returns the score and selected items maximizing the score
# 0-1 knapsack problem
def solution(capacity, catalog):
    n = len(catalog)

    # Calculate maximum score
    @lru_cache(maxsize=(capacity+1)*(n+1))
    def solve(capacity, which):
        # Selection before exceeded the capacity
        if capacity < 0:
            return -INF

        # Considred all items in catalog
        if which == n:
            return 0

        item = catalog[which]
        take_this = item.urgency + solve(capacity - item.volume, which + 1)
        skip_this = solve(capacity, which + 1)
        return max(take_this, skip_this)

    selected_items = []

    # Put selected items' name into :selected_items
    def reconstruct(capacity, which):
        if capacity < 0 or which == n:
            return

        item = catalog[which]
        if solve(capacity, which) == \
                item.urgency + solve(capacity - item.volume, which + 1):
            selected_items.append(item.name)
            reconstruct(capacity - item.volume, which + 1)
        else:
            reconstruct(capacity, which + 1)

    # Calculate results
    score = solve(capacity, 0)
    reconstruct(capacity, 0)
    return score, selected_items


C = int(input())
for _ in range(C):
    N, W = list(map(int, input().split()))
    catalog = []
    for _ in range(N):
        name, volume, urgency = input().split()
        item = Item(name, int(volume), int(urgency))
        catalog.append(item)

    score, selected_items = solution(W, catalog)
    print(score, len(selected_items))
    for item in selected_items:
        print(item)

In [ ]:
"""
Version 2:
- used methods: DP with optimization

"""
NAME, VOLUME, URGENCY = 0, 1, 2

cache = [[None] * 101 for _ in range(1001)]

catalog = None


# Calculate maximum score
def solve(capacity, which):
    # Considred all items in catalog
    if which == len(catalog):
        return 0

    ret = cache[capacity][which]
    if ret is not None:
        return ret

    item = catalog[which]
    ret = solve(capacity, which + 1)
    if capacity >= item[VOLUME]:
        ret = max(ret, item[URGENCY] + solve(capacity - item[VOLUME], which + 1))  # noqa: E501

    cache[capacity][which] = ret
    return ret


# Put selected items' name into :selected_items
def reconstruct(capacity, which, selected_items):
    if which == len(catalog):
        return

    item = catalog[which]
    if solve(capacity, which) == solve(capacity, which + 1):
        reconstruct(capacity, which + 1, selected_items)
    else:
        selected_items.append(item[NAME])
        reconstruct(capacity - item[VOLUME], which + 1, selected_items)


C = int(input())
for _ in range(C):
    N, W = list(map(int, input().split()))
    catalog = [None]*N
    for i in range(N):
        name, volume, urgency = input().split()
        catalog[i] = [name, int(volume), int(urgency)]

    # Clear cache
    for i in range(W + 1):
        for j in range(N + 1):
            cache[i][j] = None

    # Output
    score, selected_items = solve(W, 0), []
    reconstruct(W, 0, selected_items)
    print(score, len(selected_items))
    for item in selected_items:
        print(item)

In [4]:
"""
Testing

"""
import cProfile
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    ((10, [
        Item('laptop', 4, 7),
        Item('camera', 2, 10),
        Item('xbox', 6, 6),
        Item('grinder', 4, 7),
        Item('dumbell', 2, 5),
        Item('encyclopedia', 10, 4),
    ]),
     (24, ['laptop', 'camera', 'grinder'])),
    ((17, [
        Item('laptop', 4, 7),
        Item('camera', 2, 10),
        Item('xbox', 6, 6),
        Item('grinder', 4, 7),
        Item('dumbell', 2, 5),
        Item('encyclopedia', 10, 4),
    ]),
     (30, ['laptop', 'camera', 'xbox', 'grinder'])),
    ((1000, [
        Item('item-' + str(i), 10, 0) for i in range(100)
    ]),
     (0, ['item-' + str(i) for i in range(100)])),
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00014019] "solution(10, [Item(name='laptop', volume=4, urgency=7), Item(name='camera', volume=2, urgency=10), Item(name='xbox', vo... → "(24, ['laptop', 'camera', 'grinder'])"
Case 1 PASS

[0.00016332] "solution(17, [Item(name='laptop', volume=4, urgency=7), Item(name='camera', volume=2, urgency=10), Item(name='xbox', vo... → "(30, ['laptop', 'camera', 'xbox', 'grinder'])"
Case 2 PASS

[0.00808525] "solution(1000, [Item(name='item-0', volume=10, urgency=0), Item(name='item-1', volume=10, urgency=0), Item(name='item-2... → "(0, ['item-0', 'item-1', 'item-2', 'item-3', 'item-4', 'ite...
Case 3 PASS


All test(s) have passed


In [ ]:
!pip install matplotlib
"""
Performance analysis
- x-axis: n
- y-axis: O(f(n))

"""
import matplotlib as plt

_